In [67]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_classic.retrievers.parent_document_retriever import ParentDocumentRetriever
from langchain_classic.storage import InMemoryStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [68]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEy = os.getenv("OPENAI_API_KEY")

In [69]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=200)

In [70]:
# Load pdf
pdf_directory = "os-sertoes.pdf"

loader = PyPDFLoader(pdf_directory, extract_images=False)
pages = loader.load_and_split()

In [71]:
len(pages)

658

In [72]:
# Splitters

child_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True,
    #separators=["\n\n", "\n", ".", "?", "!"]
)

In [73]:
# Foi necessário fazer de uma forma diferente pq os tokens são passados de uma única vez
# estourando o máximo de tokens permitidos
parents = parent_splitter.split_documents(pages)
children = child_splitter.split_documents(parents)

print(parents)

[Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2014-05-09T00:33:41-03:00', 'moddate': '2014-05-09T00:34:14-03:00', 'trapped': '/False', 'source': 'os-sertoes.pdf', 'total_pages': 658, 'page': 0, 'page_label': 'i', 'start_index': 0}, page_content='Os sertões\nEuclides da Cunha\n1 • Os sertoes - BBB - 3ª PROVA.indd   1 09/05/14   00:33'), Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2014-05-09T00:33:41-03:00', 'moddate': '2014-05-09T00:34:14-03:00', 'trapped': '/False', 'source': 'os-sertoes.pdf', 'total_pages': 658, 'page': 1, 'page_label': 'ii', 'start_index': 0}, page_content='América Latina: a pátria grande\nDarcy Ribeiro\nPrefácio: Eric Nepomuceno\n1 • Os sertoes - BBB - 3ª PROVA.indd   2 09/05/14   00:33'), Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2014-05

In [ ]:
# Storage
store = InMemoryStore()
vectorDb = Chroma(persist_directory="naiveDB3", embedding_function=embedding)

In [ ]:
# para este caso o token estava estourando pq o add_documents manda um token gigante
# neste caso eu quebrei em batchs menores
def batch_documents(docs, batch_size):
    for i in range(0, len(docs), batch_size):
        yield docs[i:i + batch_size]


parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorDb,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)


batch_size = 100 # Ajuste este valor com base no limite da API e no tamanho dos seus documentos
for batch in batch_documents(pages, batch_size):
    parent_document_retriever.add_documents(batch)


In [79]:
TEMPLATE = """
    Use o contexto abaixo para responder à pergunta do usuário. 
    Se a resposta não estiver no contexto, diga: "Não encontrei essa informação no material fornecido."

    Query:
    {question}

    Contexto:
    {context}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [80]:
setup_retrieval = RunnableParallel({
    "question": RunnablePassthrough(),
    "context": parent_document_retriever | format_docs
})

output_parser = StrOutputParser()

In [81]:
parent_chain_retrieval = setup_retrieval | rag_prompt | llm | output_parser

In [82]:
parent_chain_retrieval.invoke("Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?")

'Euclides da Cunha apresenta uma visão complexa do ambiente natural do sertão nordestino, destacando as inter-relações entre as condições geológicas, topográficas e climáticas. Ele descreve o sertão como uma região marcada por um "conflito feito num círculo vicioso indefinido" em que as características naturais e os agentes físicos moldam a vida local de maneira intrínseca. O clima, por exemplo, é um objeto de análise que influencia diretamente a experiência dos habitantes, possibilitando entender a "significação mesológica do local".\n\nAlém disso, a obra "Os Sertões" é estruturada com foco em três elementos interligados: "A terra", "O homem" e "A luta". Essa estrutura não apenas reflete as condições naturais, mas também como essas condições afetam e fazem parte da luta e da resistência dos habitantes do sertão. Euclides ressalta que o sertanejo, típico dessa região, é visto como "'

In [83]:
parent_chain_retrieval.invoke("Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?")

'As principais características da população sertaneja, conforme descritas por Euclides da Cunha, incluem uma mistura complexa de etnias, resultando em uma mestiçagem generalizada, e uma adaptação peculiar ao ambiente árido e hostil dos sertões. Ele observa que, à medida que se aprofunda no sertão, os aspectos físicos e morais da população tendem a se uniformizar, revelando o que ele chama de "rocha viva", que representa a resistência e a força do povo sertanejo. Esta estrutura da população é vista por Euclides como influenciada tanto pelo meio ambiente quanto pela história, ressaltando a interdependência entre o ambiente e os traços culturais e sociais da população.\n\nEle relaciona essas características da população sertaneja ao seu ambiente ressaltando que as condições geológicas e climáticas moldam a sociedade local, criando um "círculo vicioso" onde o meio e o homem mutuamente se influenciam.'

In [84]:
parent_chain_retrieval.invoke("Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?")

'A Guerra de Canudos foi impulsionada por um contexto histórico e político marcado por tensões entre monarquistas e republicanos no Brasil, especialmente após a Proclamação da República em 1889. Euclides da Cunha destaca que a comunidade de Canudos, formada ao redor do beato Antônio Conselheiro, foi considerada uma ameaça às instituições republicanas, sendo acusada de representar uma reação monárquica. O governo de Floriano Peixoto, que se seguiu à república, viu a necessidade de eliminar essa indignação e resistência popular, levando ao envio de expedições militares com a intenção de destruir Canudos, vista como um símbolo de desobediência e resistência contra o novo regime. Além disso, o comprometimento dos militares e suas derrotas nas primeiras expedições alimentaram uma narrativa de combate à suposta rebelião, resultando em uma campanha brutal que culminou na destruição de Belo Monte.'

In [85]:
parent_chain_retrieval.invoke("Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?")

'Euclides da Cunha descreve Antônio Conselheiro como uma figura carismática e quase mística, que goza de grande influência entre seus seguidores, a ponto de serem descritos como adoradores que o tratam como um "Deus vivo". Ele é apresentado como um "apóstolo desnorteado", que persegue uma causa religiosa e social, construindo uma capela em Canudos com o apoio financeiro de seus fiéis, que vendem até o que possuem para sustentá-lo. Conselheiro formou uma comunidade em Belo Monte, que representa uma forma de resistência contra as autoridades estabelecidas. A partir de 1896, suas atividades geraram temores e boatos que culminaram nas expedições militares do governo. Para Euclides, a guerra em Canudos não é apenas um conflito militar, mas uma expressão de uma série de injustiças sociais e um clamor por reconhecimento e respeito por aquelas comunidades.'

In [86]:
parent_chain_retrieval.invoke("Quais são os principais aspectos da crítica social e política presentes em 'Os Sertões'? Como esses aspectos refletem a visão do autor sobre o Brasil da época?")

'Os principais aspectos da crítica social e política presentes em "Os Sertões" incluem a análise da realidade geográfica e humana do sertão, a crítica ao preconceito em relação ao sertanejo e a reflexão sobre as transformações sociais do Brasil no final do século XIX. A obra retrata o sertanejo como "a rocha viva da nossa raça", defendendo sua resistência e complexidade, enquanto critica a visão do sertão como uma sociedade bárbara em contraposição à civilização litorânea.\n\nEsses aspectos refletem a visão do autor, Euclides da Cunha, sobre um Brasil em transformação, marcada pelo fim da Guerra do Paraguai e pela Abolição da Escravatura, quando diversas correntes políticas e filosóficas emergiam. A obra articula a questão da miscigenação, da identidade nacional e da luta dos deserdados, mostrando uma profunda preocupação com a condição social e a história do povo brasileiro. A narrativa, estruturada em uma forma que'